In [3]:
import torch
import numpy as np
import os
import json
import joblib
from torch.utils.data import Dataset,DataLoader
from itertools import repeat
import pandas as pd
import math
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning.loggers import CSVLogger

/common/home/vk405/miniconda3/envs/Crossmdl/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# m = nn.Sigmoid()
# loss = nn.BCELoss()
# input = torch.randn(3, requires_grad=True)
# target = torch.empty(3).random_(2)
# output = loss(m(input), target)
# output.backward()

In [5]:
# input

In [6]:
# target

In [7]:
# class tdset(Dataset):
#     def __init__(self,ids):
#         self.ids = ids
#         self.sz = np.random.randint(5,10)
#     def __len__(self):
#         return len(self.ids)
#     def __getitem__(self,idx):
#         return np.random.randn(self.sz)


In [8]:
# dset = tdset([0,9,8,6,8,4])
# dset2 = tdset([0,9,8,6,8,4])

In [9]:
# # 50 percentile length is 25 frames.Thus we will have the sequence of video-frames = 25

# class TOYMODEL(pl.LightningModule):
#     def __init__(self,hparams):
#         super().__init__()
#         self.save_hyperparameters(hparams)
#         self.lin = nn.Linear(15,1)

#     def forward(self,x):
#         return self.lin(x)

#     def training_step(self,batch,batch_idx):
#         out = self.lin(x)
        

    


In [10]:
#utils

def get_vids(base_dir,split):
    trn_split = base_dir+split
    trn_idlst = []
    trn_vidlst = []

    f = open(trn_split,'r')
    for line in f:
        id_,vid = line.split('/')
        vid = vid.strip('\n')
        trn_idlst.append(id_)
        trn_vidlst.append(vid)
        #print(vid)
        #break
    f.close()
    return trn_idlst,trn_vidlst

    
def get_features(data_dir,split='val',feat_dir='/common/users/vk405/feat_csv/'):
    #feat_dir = data_dir
    splits_dir = data_dir+'splits/'
    if split == 'val':
        feat_split_dir = feat_dir+'val_frame_feat_csv/'  
        vid_num,vid_name = get_vids(splits_dir,'val_list.txt')  
    elif split == 'train':
        feat_split_dir = feat_dir+'train_frame_feat_csv/'  
        vid_num,vid_name = get_vids(splits_dir,'train_list.txt') 
    elif split == 'test':
        feat_split_dir = feat_dir+'test_frame_feat_csv/'  
        vid_num,vid_name = get_vids(splits_dir,'test_list.txt')
    else:
        raise NotImplementedError(f'unknown split: {split}')     
    feat_list = {}
    vid_dtls = []
    for num,name in zip(vid_num,vid_name):
        feat_loc = os.path.join(feat_split_dir, f'{num}/{name}/0001/')
        #import pdb;pdb.set_trace()
        if os.path.isdir(feat_loc):
            feat_files = feat_loc + os.listdir(feat_loc)[0]
            feat_list[name] = feat_files
            #feat_list.append(feat_files)
            vid_dtls.append((num,name))
        else:
            print(f"video : {num}/{name} not found")
    assert len(feat_list) == len(vid_dtls),"get-features is giving incorrect features"
    return feat_list,vid_dtls






def get_raw_labels(ids,annotns_file):

    label_info = {}
    with open(annotns_file) as json_file:
        annotns = json.load(json_file)
        print(annotns.keys())
        for _,vidname in ids:
            #import pdb;pdb.set_trace()
            if vidname in annotns['database']:
                #import pdb;pdb.set_trace()
                duration = annotns['database'][vidname]['duration']
                annot = annotns['database'][vidname]['annotations']
                labels = []
                #import pdb;pdb.set_trace()
                for segment_info in annot:
                    interval = segment_info['segment']
                    sent = segment_info['sentence']
                    labels.append((interval,sent,duration))

                label_info[vidname] = labels
            else:
                print(f"label for {vidname} not present")
    return label_info

def regress_labels(raw_labels):
    regress_labels = {}
    for key in raw_labels:
        new_labels = []
        for item in raw_labels[key]:
            rng,sent,vidlen = item
            mid = sum(rng)/2
            duration = rng[-1]-rng[0]
            mid_pred = (1/vidlen)*mid # location of mid-point w.r.t video length
            duration_pred = (1/vidlen)*duration
            new_labels.append(([mid_pred,duration_pred],sent))
        regress_labels[key] = new_labels
    return regress_labels
            
            
    
    
    


In [11]:
from sklearn.base import BaseEstimator, TransformerMixin

class LabelEncoder2(BaseEstimator, TransformerMixin):
    def __init__(self,max_len=499):
        self.vidlens = []
        self.truebounds = []
        self.max_len = max_len

    def fit(self,raw_labels):
        l = []
        for key in raw_labels:
            vid_len = raw_labels[key][0][-1]
            sz = len(raw_labels[key])
            for i in range(sz):l.append(vid_len)
        self.vidlens = np.array(l)
        return self
        
    def transform(self,raw_labels):
        regress_labels = self._regress_labels(raw_labels)
        return regress_labels

    def decode(self,outputs):
        return np.round(outputs*self.max_len)


    def _regress_labels(self,raw_labels):
        regress_labels = {}
        bounds = []
        for key in raw_labels:
            new_labels = []
            for item in raw_labels[key]:
                #import pdb;pdb.set_trace()
                rng,sent,vidlen = item
                new_rng = [rng[0]/self.max_len,rng[-1]/self.max_len]
                bounds.append(rng)
                new_labels.append((new_rng,sent))
            regress_labels[key] = new_labels
        self.truebounds = np.array(bounds)
        return regress_labels


In [12]:

def get_labels(ids,annotns_file):

    label_info = {}
    with open(annotns_file) as json_file:
        annotns = json.load(json_file)
        #print(annotns.keys())
        for _,vidname in ids:
            #import pdb;pdb.set_trace()
            if vidname in annotns:
                #import pdb;pdb.set_trace()
                duration = annotns[vidname]['duration']
                annot = annotns[vidname]['annotations']
                labels = []
                #import pdb;pdb.set_trace()
                for segment_info in annot:
                    interval = segment_info['segment']
                    st_end = [interval[0],interval[-1]]
                    sent = segment_info['sentence']
                    labels.append((st_end,sent,duration))

                label_info[vidname] = labels
            else:
                print(f"label for {vidname} not present")
    return label_info

In [13]:
#dataset
# Dataset/loader
# This is newer version
class YoucookDset2(Dataset):
    def __init__(self,data_dir='/common/home/vk405/Projects/Crossmdl/Data/YouCookII/'\
        ,split='train',use_precomp_emb=True,seqlen=26,framecnt=499,id=0):
        self.id = id
        self.feat_locs = {}
        self.split = split
        self.data_dir = data_dir
        self.use_precomp_emb = use_precomp_emb
        self.text_emb = None
        self.seqlen = seqlen
        self.framecnt = framecnt
        if self.split != 'test':
            self.annotns_file = data_dir+'annotations/segment_youcookii_annotations_trainval.json'
        else:
            raise NotImplementedError(f"Split:{self.split},not yet correctly implemented")
        if self.use_precomp_emb:
            self.txt_emb = joblib.load(os.path.join(self.data_dir,'emb.joblib'))

        self.feat_locs,vids = get_features(self.data_dir,split=self.split)
        assert len(vids) == len(self.feat_locs),"features are wrong"
        #import pdb;pdb.set_trace()
        label_info = get_labels(vids,self.annotns_file)
        #self.labelencoder = LabelEncoder2()
        self.final_labels = label_info
        #self.labelencoder.fit_transform(label_info)
        
        #regress_labels(label_info)
        #(vid_id,seg_id)
        self.update_data()

                
            
    def __len__(self):
        return len(self.data)

    def update_data(self,id=None):
        self.data = []
        #self.vid_len = []
        if not id:
            id = self.id
        starting_pnt = np.arange(id,self.framecnt,self.seqlen)

        for key in self.final_labels:
            annot_len = len(self.final_labels[key])
            if key in self.feat_locs:
                file_loc = self.feat_locs[key]
                #for stpnt in starting_pnt:
                segments = list(zip(repeat(key,annot_len),repeat(file_loc,annot_len),\
                        range(annot_len)))
                for seg in segments:
                    for stpnt in starting_pnt:
                        if stpnt+self.seqlen<=self.framecnt:
                            datapnt = seg[:-1]+(stpnt,)+seg[-1:]
                            self.data.append(datapnt)
                    
                #self.data.extend(segments)
            else:
                print(f"video:{key} not found")


    def getclass_prob(self,lbl_rng,frame_rng):
        lbl_ids = set(np.arange(lbl_rng[0],lbl_rng[-1]+1))
        frame_ids = set(np.arange(frame_rng[0],frame_rng[-1]+1))
        inter = lbl_ids.intersection(frame_ids)
        if len(inter) == 0:
            return 0.0
        else:
            return len(inter)/len(lbl_ids.union(frame_ids))

    def __getitem__(self,idx):
        if self.use_precomp_emb:
            vidname,file_loc,stid,seg_ind = self.data[idx]
            #import pdb;pdb.set_trace()
            #self.txt_emb[vidname][seg_ind],
            txt_info = self.final_labels[vidname][seg_ind]
            label_value = self.getclass_prob(txt_info[0],(stid,stid+self.seqlen-1))
            #import pdb;pdb.set_trace()
            return pd.read_csv(file_loc).values.astype(np.float32)[stid:stid+self.seqlen,:],(self.txt_emb[vidname][seg_ind]).astype(np.float32),\
                label_value
            #np.array(self.final_labels[vidname][seg_ind][0],dtype=np.float32)
        else:
            raise NotImplementedError("not yet correctly implemented")

        

           



        

In [14]:

# data_dir = '/common/home/vk405/Projects/Crossmdl/Data/YouCookII/'



# youcookdata = YoucookDset2()

In [15]:
# youcookdata[19][0].shape

In [16]:
# #load dataset and dataloader once every epoch, makes stuff a bit slow but this is a simple approach.


# youcookdata[19][-1]

In [17]:
# youcookdata.final_labels['sdB8qBlLS2E']

In [18]:
# youcookdata[0][-1]

In [19]:
# youcookdata[1][-1]

In [20]:
#model utils

#!pip install transformers

def init_parameters_xavier_uniform(model):
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

def scaled_dot(query, key, mask_key=None):  
    score = torch.matmul(query, key.transpose(-2, -1))
    score /= math.sqrt(query.size(-1))
    if mask_key is not None:
        score = score.masked_fill(mask_key, -1e18)  # Represents negative infinity
    return score      
            
def attend(query, key, value, mask_key=None, dropout=None):
    # TODO: Implement
    # Use scaled_dot, be sure to mask key
    #smax = nn.Softmax(-1)
    #import pdb;pdb.set_trace()
    score = scaled_dot(query,key,mask_key)  
    attention = F.softmax(score,dim=-1)
    if dropout is not None:#do = nn.Dropout(dropout)
        attention = dropout(attention)
    answer = torch.matmul(attention,value) 
    # Convexly combine value embeddings using attention, this should be just a matrix-matrix multiplication.
    return answer, attention



def split_heads(batch, num_heads):  
    (batch_size, length, dim) = batch.size()  # These are the expected batch dimensions.
    assert dim % num_heads == 0  # Assert that dimension is divisible by the number of heads.
    dim_head = dim // num_heads

    # No new memory allocation
    splitted = batch.view(batch_size, -1, num_heads, dim_head).transpose(1, 2)  
    return splitted  # (batch_size, num_heads, length, dim_head), note that now the last two dimensions are compatible with our attention functions. 




def merge_heads(batch):  
    (batch_size, num_heads, length, dim_head) = batch.size()  # These are the expected batch dimensions.

    # New memory allocation (reshape), can't avoid.
    merged = batch.transpose(1, 2).reshape(batch_size, -1, num_heads * dim_head)
    return merged  # (batch_size, length, dim)


class MultiHeadAttention(nn.Module):
    def __init__(self, dim, num_heads, dropout_rate=0.1):
        super().__init__()
        assert dim % num_heads == 0

        self.linear_query = nn.Linear(dim, dim)
        self.linear_key = nn.Linear(dim, dim)
        self.linear_value = nn.Linear(dim, dim)
        self.linear_final = nn.Linear(dim, dim)
        self.dropout = nn.Dropout(dropout_rate)

        self.num_heads = num_heads

    def forward(self, query, key, value, mask_key=None, layer_cache=None,
              memory_attention=False):
        """
        INPUT
          query: (batch_size, length_query, dim)
          key: (batch_size, length_key, dim)
          value: (batch_size, length_key, dim_value)
          mask_key: (*, 1, length_key) if queries share the same mask, else
                    (*, length_query, length_key)
          layer_cache: if not None, stepwise decoding (cache of key/value)
          memory_attention: doing memory attention in stepwise decoding?
        OUTPUT
          answer: (batch_size, length_query, dim_value)
          attention: (batch_size, num_heads, length_query, length_key) else
        """
        batch_size = query.size(0)

        query = self.linear_query(query)
        query = split_heads(query, self.num_heads)  # (batch_size, num_heads, -1, dim_head)

        def process_key_value(key, value):  # Only called when necessary.
            key = self.linear_key(key)
            key = split_heads(key, self.num_heads)
            value = self.linear_value(value)
            value = split_heads(value, self.num_heads)
            return key, value

        #import pdb;pdb.set_trace()
        if layer_cache is None:
            key, value = process_key_value(key, value)
        else:
            assert query.size(2) == 1  # Stepwise decoding
            
            if memory_attention:
                if layer_cache['memory_key'] is None:  # One-time calculation
                    key, value = process_key_value(key, value)
                    # (batch_size, num_heads, length_memory, dim)
                    layer_cache['memory_key'] = key
                    layer_cache['memory_value'] = value

                key = layer_cache['memory_key']
                value = layer_cache['memory_value']

            else:  # Self-attention during decoding
                key, value = process_key_value(key, value)
                assert key.size(2) == 1 and value.size(2) == 1
                
                # Append to previous.
                if layer_cache['self_key'] is not None:
                    key = torch.cat((layer_cache['self_key'], key), dim=2)
                    value = torch.cat((layer_cache['self_value'], value), dim=2)
                    
                 # (batch_size, num_heads, length_decoded, dim)
                layer_cache['self_key'] = key  # Recache.
                layer_cache['self_value'] = value
        # Because we've splitted embeddings into heads, we must also split the mask. 
        # And because each query uses the same mask for all heads (we don't use different masking for different heads), 
        # we can specify length 1 for the head dimension.
        if mask_key is not None:  
            mask_key = mask_key.unsqueeze(1)  # (batch_size, 1, -1, length_key)

        answer, attention = attend(query, key, value, mask_key, self.dropout)

        answer = merge_heads(answer)  # (batch_size, length_key, dim)
        answer = self.linear_final(answer)

        return answer, attention

class PositionwiseFeedForward(nn.Module):
    def __init__(self, dim, dim_hidden, drop_rate=0.1):
        super().__init__()
        self.w1 = nn.Linear(dim, dim_hidden)
        self.w2 = nn.Linear(dim_hidden, dim)
        self.layer_norm = nn.LayerNorm(dim, eps=1e-6)
        self.drop1 = nn.Dropout(drop_rate)
        self.relu = nn.ReLU()
        self.drop2 = nn.Dropout(drop_rate)
    def forward(self, x):
        inter = self.drop1(self.relu(self.w1(self.layer_norm(x))))
        output = self.drop2(self.w2(inter))
        return output + x




class SinusoidalPositioner(nn.Module):
    def __init__(self, dim, drop_rate=0.1, length_max=5000):
        super().__init__()
        frequency = torch.exp(torch.arange(0, dim, 2) * -(math.log(10000.) / dim))  # Using different frequency for each dim
        positions = torch.arange(0, length_max).unsqueeze(1)
        wave = torch.zeros(length_max, dim)
        wave[:, 0::2] = torch.sin(frequency * positions)
        wave[:, 1::2] = torch.cos(frequency * positions)
        self.register_buffer('wave', wave.unsqueeze(0))  # (1, length_max, dim)
        self.dropout = nn.Dropout(drop_rate)
        self.dim = dim
        self.length_max = length_max
    def forward(self, x, step=-1):
        assert x.size(-2) <= self.length_max

        if step < 0:  # Take the corresponding leftmost embeddings.
            position_encoding = self.wave[:, :x.size(-2), :]
        else:  # Take the embedding at the step.
            position_encoding = self.wave[:, step, :]

        x = x * math.sqrt(self.dim)
        return self.dropout(x + position_encoding)




class TransformerEncoderLayer(nn.Module):

  def __init__(self, dim, num_heads, dim_hidden, drop_rate):
    super().__init__()
    self.layer_norm = nn.LayerNorm(dim, eps=1e-6)
    self.self_attention = MultiHeadAttention(dim, num_heads, drop_rate)
    self.drop = nn.Dropout(drop_rate)
    self.feedforward = PositionwiseFeedForward(dim, dim_hidden, drop_rate)

  def forward(self, source, mask_source=None):
    # TODO: Implement
    #print(source.shape)
    normed = self.layer_norm(source)  
    # Apply layer norm on source

    attended, attention = self.self_attention(normed,normed,normed,mask_source)
    #None, None  # Apply self-attention on normed (be sure to use mask_source).
    attended = self.drop(attended) + source  
    # Re-write attended by applying dropout and adding a residual connection to source.
    return self.feedforward(attended), attention




class CrossAttentionLayer(nn.Module):
    def __init__(self,dim,num_heads,dim_hidden,drop_rate):
        super().__init__()
        self.layer_norm = nn.LayerNorm(dim, eps=1e-6)
        self.context_attention = MultiHeadAttention(dim, num_heads, drop_rate)
        self.drop = nn.Dropout(drop_rate)
        self.feedforward = PositionwiseFeedForward(dim, dim_hidden, drop_rate)
        
    def forward(self,target,memory,layer_cache=None):
        
        cross_attn_target = self.layer_norm(target)
        attended, attention = self.context_attention(cross_attn_target,memory,memory,layer_cache=layer_cache,memory_attention=True)
        
        attended = target + self.drop(attended)
        
        return self.feedforward(attended),attention



layer_cache = {'memory_key': None, 'memory_value': None, 'self_key': None, 'self_value': None}

In [21]:
#model 






class CrossattnModel(pl.LightningModule):
    def __init__(self,hparams,dset=None):
        super().__init__()
        self.save_hyperparameters(hparams)
        #self.hparams = hparams
        #import pdb;pdb.set_trace()
        #self.net= Model(hparams)
        #self.hparams  = hparams
        self.positioner = SinusoidalPositioner(self.hparams.edim, drop_rate=0., length_max=1000)
        self.attn = CrossAttentionLayer(self.hparams.edim,self.hparams.nheads,\
                           self.hparams.attnhdim,self.hparams.dropoutp)
        self.wrdcnn =  nn.Conv1d(self.hparams.wrdim, self.hparams.edim, 1, stride=1)
        self.vidcnn =  nn.Conv1d(self.hparams.vidim, self.hparams.edim, 1, stride=1)
        self.hid_layer = nn.Linear(self.hparams.edim,self.hparams.hdim)
        self.out_layer = nn.Linear(self.hparams.hdim,1)
        #self.init_parameters_xavier_uniform()
        self.dset = dset

    def forward(self,x):
        #keep this for inference
        out = self.net(x)
        return out
        
    def net(self,x):
        vid_x,wrd_x = x
        #import pdb;pdb.set_trace()
        wrd_x = wrd_x.unsqueeze(1).transpose(1,2)
        vid_x = vid_x.transpose(1,2)
        #print(f"inside model, wrd_x:{wrd_x.shape},vi")
        tgt = self.wrdcnn(wrd_x.float()).transpose(1,2)
        src = self.vidcnn(vid_x.float()).transpose(1,2)
        src_posencode = self.positioner(src)
        #for i in range(self.hparams.lyrs):
        attended,attn_score = self.attn(tgt,src_posencode)
            #tgt = 
        out = F.sigmoid(self.out_layer(F.relu(self.hid_layer(F.relu(attended)))))
        return out



    def training_step(self,batch,batch_idx):
        #for tranining
        vid_feat,wrd_feat,labels = batch
        x_hat = self.net((vid_feat.float(),wrd_feat.float()))
        #import pdb;pdb.set_trace()
        #loss = nn.BCELoss()
        loss = F.binary_cross_entropy(x_hat.squeeze().float(), labels.squeeze().float())
        #print(f"inside train step, loss:{loss}")
        self.log("train_loss",loss,on_step=True)
        return loss

    def validation_step(self,batch,batch_idx):
        #for validation
        vid_feat,wrd_feat,labels = batch
        x_hat = self.net((vid_feat.float(),wrd_feat.float()))
        #import pdb;pdb.set_trace()
        loss = F.binary_cross_entropy(x_hat.squeeze().float(), labels.squeeze().float())
        #print(f"inside train step, loss:{loss}")
        self.log("val_loss",loss,on_step=False, on_epoch=True)
        return loss


    def configure_optimizers(self):
        lr = self.hparams.lr if 'lr' in self.hparams else 1e-3
        optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        return optimizer

    
    def init_parameters_xavier_uniform(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def train_dataloader(self):
        ind = self.current_epoch % self.hparams.seqlen
        self.dset.id = ind
        self.dset.update_data()
        print(f"train dataloader called with ind:{ind}")
        return DataLoader(self.dset,self.hparams.batch_size,shuffle=True,num_workers=10)
        



In [22]:
#seqlen=25,framecnt=500,id=0

In [23]:
# log_dir = '/common/home/vk405/Projects/Crossmdl/nbs/lightning_logs/'

# from pytorch_lightning.loggers import CSVLogger
# version = '3'
# csvlogger = CSVLogger(log_dir,version)
# #trainer = pl.Trainer(logger= csvlogger,\
#     #s=1gpu,max_epochs=15)
# #reload_dataloaders_every_epoch=True
# #youcoodvld_dl = DataLoader(youcookvld2,batch_size=64,shuffle=False,num_workers=10)

In [24]:
# #trainer.fit(model,youcookdl)
# #model = CrossattnModel(hparams)

# youcookdata = YoucookDset2()


# import torch.utils.data as data_utils
# #indices = torch.arange(1000)
# #youcookdatasubset = data_utils.Subset(youcookdata, indices)
# youcookdl = DataLoader(youcookdata,batch_size=128,shuffle=True,num_workers=20)



# model = CrossattnModel(hparams,youcookdata)


In [25]:
#trainer.fit(model,youcookdl)

In [26]:
# # similar, but with a fixed 10 batches no matter the size of the dataset
# #trainer.fit(model,youcookdl)
# #sanity check
# version = '5'
# log_dir = '/common/home/vk405/Projects/Crossmdl/nbs/lightning_logs/'
# csvlogger = CSVLogger(log_dir,version)
# trainer = pl.Trainer(logger=csvlogger,gpus=1,accelerator="gpu",auto_scale_batch_size="binsearch")
# trainer.tune(model)

# #trainer.fit(model,youcookdl)

In [37]:
#4096 works
import wandb
import logging
from pytorch_lightning.loggers import CSVLogger, TensorBoardLogger, WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import os

logger = logging.getLogger(__name__)
wandb_logger = lambda dir, version: WandbLogger(
    name="wandb", save_dir=dir, version=version
)
csvlogger = lambda dir, version: CSVLogger(dir, name="csvlogs", version=version)
tblogger = lambda dir, version: TensorBoardLogger(dir, name="tblogs", version=version)

def get_loggers(dir,version,lis=["csv"]):
    lgrs = []
    if "wandb" in lis:
        lgrs.append(wandb_logger(dir, version))
    if "csv" in lis:
        lgrs.append(csvlogger(dir, version))
    if "tb" in lis:
        lgrs.append(tblogger(dir, version))
    return lgrs





In [38]:
def run(cfg):
    pl.seed_everything(cfg.seed)
    dir = cfg.artifacts_loc
    version = str(cfg.version)
    logger_list = get_loggers(dir, version,cfg.loggers)
    cbs = []
    if "early_stop" in cfg.cbs:
        #? does'nt really work atm
        params = cfg.model.cbs.early_stop
        earlystopcb = EarlyStopping(**params, min_delta=0.00, verbose=False)
        cbs.append(earlystopcb)
    if "checkpoint" in cfg.cbs:
        store_path = dir + "ckpts/" + str(cfg.version) + "/"
        isExist = os.path.exists(store_path)
        if not isExist:
            os.makedirs(store_path)
        fname = "{epoch}-{train_loss:.2f}"
        params = cfg.checkpoint
        checkptcb = ModelCheckpoint(**params, dirpath=store_path, filename=fname)
        cbs.append(checkptcb)

    wandb.init(project="videoretrieval", config=cfg)
    if cfg.mode == 'train':
        youcookdata = YoucookDset2(data_dir=cfg.data_dir,split=cfg.mode,\
               use_precomp_emb=cfg.use_precomp_emb,\
                   seqlen=cfg.seqlen,framecnt=cfg.framecnt,id=cfg.id)
        train_loader = DataLoader(youcookdata,\
            batch_size=cfg.batch_size,shuffle=True,num_workers=4,pin_memory=True)
        hparams = cfg    
        net = CrossattnModel(hparams,youcookdata)
        trainer = pl.Trainer(
            logger=logger_list,callbacks=cbs, gpus=1,deterministic=True, **cfg.trainer
        )
        trainer.fit(net, train_loader)
        return trainer
        #trainer.tune(net,train_loader)
            
    else:
        pass
    

In [39]:
from argparse import Namespace
cfg = Namespace(
    version = 'trail_again',
    id = 0,
    artifacts_loc = "/common/home/vk405/Projects/Crossmdl/nbs/",
    data_dir = "/common/home/vk405/Projects/Crossmdl/Data/YouCookII/",
    mode = 'train',
    loggers = ["csv","wandb"],
    seed = 0,
    cbs = ["checkpoint"],
    trainer = {'log_every_n_steps': 1,
    'max_epochs': 10},
    checkpoint = {"every_n_epochs": 1,
    "monitor": "train_loss"},

    use_precomp_emb = True,
    edim = 100,
    attnhdim = 50,
    nheads = 10,
    wrdim = 768,
    vidim = 512,
    hdim = 30,
    dropoutp=0.0,
    seqlen=26,
    framecnt=499,
    batch_size=512


)

In [40]:
traned_model = run(cfg)

Global seed set to 0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vin136 (use `wandb login --relogin` to force relogin)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/common/home/vk405/miniconda3/envs/Crossmdl/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:122: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
/common/home/vk405/miniconda3/envs/Crossmdl/lib/python3.8/site-packages/pytorch_lightning/loggers/wandb.py:341: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
/common/home/vk405/miniconda3/envs/Crossmdl/lib/python3.8/site-packages/pytorch_lightning/loggers/csv_logs.py:57: UserWarning: Experiment logs directory /common/home/vk405/Projects/Cross

Epoch 0:   1%|          | 3/384 [00:30<1:05:15, 10.28s/it, loss=0.527, v_num=gain]

/common/home/vk405/.local/lib/python3.8/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch 0:   2%|▏         | 7/384 [01:01<55:03,  8.76s/it, loss=0.338, v_num=gain]  

In [32]:

youcookdata = YoucookDset2(data_dir=cfg.data_dir,split=cfg.mode,\
               use_precomp_emb=cfg.use_precomp_emb,\
                   seqlen=cfg.seqlen,framecnt=cfg.framecnt,id=cfg.id)
net = CrossattnModel(cfg,youcookdata)

In [31]:
PATH = '/common/home/vk405/Projects/Crossmdl/nbs/ckpts/trail_again/epoch=0-train_loss=0.18.ckpt'

#net.load_state_dict(torch.load(PATH))

new_model = net.load_from_checkpoint(checkpoint_path=PATH)

In [30]:
youcookdata = YoucookDset2(data_dir=cfg.data_dir,split=cfg.mode,\
               use_precomp_emb=cfg.use_precomp_emb,\
                   seqlen=cfg.seqlen,framecnt=cfg.framecnt,id=cfg.id)

In [31]:
# I made a blunder-> dataset is not balanced.
best_fits = []
labels = []
max_len = 10
for a in youcookdata:
    vid_em,txt_emb,label = a
    #import pdb;pdb.set_trace()
    #print(label)
    if label>=0.6:
        best_fits.append((vid_em,txt_emb))
        labels.append(label)
    if len(labels)>max_len:
        break

In [44]:
train_loader = DataLoader(youcookdata,\
            batch_size=cfg.batch_size,shuffle=True,num_workers=4,pin_memory=True)

torch.Size([512, 26, 512])
torch.Size([512, 768])


In [47]:
sampl_inpt[1].shape

torch.Size([1, 26, 512])

In [53]:
preds_list = []
for i in range(len(labels)):
    sampl_inpt = (torch.tensor(best_fits[i][0]).unsqueeze(axis=0),torch.tensor(best_fits[i][1]).unsqueeze(0))
    pred_sampl = net(sampl_inpt)
    out = pred_sampl.detach().item()
    preds_list.append(out)
    

/common/home/vk405/.local/lib/python3.8/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [54]:
preds_list

[0.9124129414558411,
 0.8083134889602661,
 0.9570930004119873,
 0.9850350618362427,
 0.36355796456336975,
 0.9555460810661316,
 0.6632380485534668,
 0.7027997374534607,
 0.9942954182624817,
 0.9886470437049866,
 0.9781237244606018]

In [55]:
# I made a blunder-> dataset is not balanced.
best_fits = []
labels = []
max_len = 10
for a in youcookdata:
    vid_em,txt_emb,label = a
    #import pdb;pdb.set_trace()
    #print(label)
    if label<=0.1:
        best_fits.append((vid_em,txt_emb))
        labels.append(label)
    if len(labels)>max_len:
        break

In [56]:
preds_list = []
for i in range(len(labels)):
    sampl_inpt = (torch.tensor(best_fits[i][0]).unsqueeze(axis=0),torch.tensor(best_fits[i][1]).unsqueeze(0))
    pred_sampl = net(sampl_inpt)
    out = pred_sampl.detach().item()
    preds_list.append(out)
    

/common/home/vk405/.local/lib/python3.8/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [57]:
preds_list

[0.7997788190841675,
 0.9861284494400024,
 0.9484415650367737,
 0.9937611222267151,
 0.9359134435653687,
 0.911560595035553,
 0.9371639490127563,
 0.9650371074676514,
 0.9784882068634033,
 0.9068688750267029,
 0.9708224534988403]

In [52]:
pred_sampl.detach().item()

0.9124129414558411

In [67]:
# I made a blunder-> dataset is not balanced.
best_fits = []
labels = []
max_len = 100
preds_list = []
for a in youcookdata:
    vid_em,txt_emb,label = a
    #import pdb;pdb.set_trace()
    #print(label)
    sampl_inpt = (torch.tensor(vid_em).unsqueeze(axis=0),torch.tensor(txt_emb).unsqueeze(0))
    pred_sampl = net(sampl_inpt)
    out = pred_sampl.detach().item()
    preds_list.append(out)
    labels.append(label)
    if len(labels)>max_len:
        break

In [68]:
#labels

In [33]:
#get a label distribution

from tqdm import tqdm
# I made a blunder-> dataset is not balanced.
#best_fits = []
labels = []
max_len = 2000
for a in tqdm(youcookdata):
    vid_em,txt_emb,label = a
    #import pdb;pdb.set_trace()
    #print(label)
    #best_fits.append((vid_em,txt_emb))
    labels.append(label)
    if len(labels)>max_len:
        break

    

  1%|          | 2000/196403 [02:11<3:33:20, 15.19it/s]


In [35]:
df = pd.DataFrame({'percentile_ind':np.linspace(0,100,25),
 'percentile':[np.percentile(labels,p) for p in np.linspace(0,100,25)]})

In [36]:
#distribution is highly skewed
df

,percentile_ind,percentile
0,0.000000,0.000000
1,4.166667,0.000000
2,8.333333,0.000000
3,12.500000,0.000000
4,16.666667,0.000000
5,20.833333,0.000000
6,25.000000,0.000000
7,29.166667,0.000000
8,33.333333,0.000000
9,37.500000,0.000000


[0.0,
 0.0,
 0.0,
 0.19230769230769232,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.423728813559322,
 0.4482758620689655,
 0.09090909090909091,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.7307692307692307,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5,
 0.26666666666666666,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.23076923076923078,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.14583333333333334,
 0.6666666666666666,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.38461538461538464,
 0.3170731707317073,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0

In [32]:
#mytrainer = run(cfg)
#'epoch=0-train_loss=0.18.ckpt'

In [33]:
# pl.seed_everything(cfg.seed)
# dir = cfg.artifacts_loc
# version = str(cfg.version)
# logger_list = get_loggers(dir, version,cfg.loggers)

# youcookdata = YoucookDset2(data_dir=cfg.data_dir,split=cfg.mode,\
#                use_precomp_emb=cfg.use_precomp_emb,\
#                    seqlen=cfg.seqlen,framecnt=cfg.framecnt,id=cfg.id)

# train_loader = DataLoader(youcookdata,\
# batch_size=cfg.batch_size,shuffle=True,num_workers=4,pin_memory=True)
# hparams = cfg    
# net = CrossattnModel(hparams,youcookdata)
# trainer = pl.Trainer(
# logger=logger_list, auto_lr_find=True,callbacks=[], gpus=1,deterministic=True, **cfg.trainer
# )
# trainer.fit(net, train_loader)
# #trainer.tune(net,train_loader)

# # Run learning rate finder
# lr_finder = trainer.tuner.lr_find(net,train_loader)



In [34]:
# # Results can be found in
# lr_finder.results

# # Plot with
# fig = lr_finder.plot(suggest=True)
# fig.show()

In [35]:
# lr_finder.results.keys()